In [1]:
df = read_csv(f"all_v_all_alignments.csv").set_index("sequence")

NameError: name 'read_csv' is not defined

In [ ]:
save("seqIDs.npy", df.columns)

In [ ]:
save("sequences.npy", df.index.to_numpy())

In [ ]:
df

,3d4d0abe98096fcaed17decc09dc6c4c,2d8d5004dc994f9afa3a48bc723b650b,f8c9c4f42f6641d5e0f5b67ff8bb8964,957b8233c0cbc03c32413da7114d500e,4fa86fad10569d967e171843206742c4,13a64a12959741e1ec2e9590a8693391,8466c6dfab56e8746ea47bc3c0e571e0,75684db37a815fbf97b5e847e11dbc73,3650c400f224650eba1d59c8807f6a87,6c7cc25ccfb317886a6628773d347cde,...,b2f5f7bc040d7a596b4014f46f888d6c,70e46338ba5623f7e8a9955c741f4c4b,872e13dbf301a9540ef3ed2792c063ce,48a3247653308c6015d308fb05fee544,de466fe1506162725f86228b26a130c0,5313a460934f51b9af6019824504178c,48a7ab8e593ebe5f41034be027c6be7e,25ebb445f83e61423a85a00a877e071c,ee06df355dbbfb9569e33b9e88558664,345d5ab1fe0a8fb8e2a2e11058fe5594
sequence,,,,,,,,,,,,,,,,,,,,,
AACGTAGGTGGCGAGCGTTATCCGGATTTACTGGGCGTAAAGAGCGCGTAGGTGGTTGAGTAAGTTGGATGTAAAATCTCTTGGCTTAACTGGGAGGAGACGTTCAAGACTGCTTGGCTTGAGGGCGAGAGAGGGGTGCAGAATTCCCGGTGTAGTGGTGGAATGCGTAGATATCGGGAGGAATACCAGTGGCGAAAGCGGCGCCCTGGCTCGCAACTGACACTGAGGCGCGAAAGCGTGGGTAGCGAACGGGATTAGATACCCCGGTAGTCCACGCTGTAAACGATGTGAACTGGGTGTTGGCGGTATGAATTCCGTCGGTGCCGTAGCAAACGCGATAAGTTCACCGCCTGGGGAGTACGGTCGCAAGGCTA,0.709893,0.816845,0.739305,0.687166,0.819519,0.672460,0.782086,0.691176,0.743316,0.763369,...,0.675134,0.695187,0.651070,0.709893,0.754011,0.748663,0.783422,0.788770,0.740642,0.679144
NaN,-45.375000,-46.375000,-46.250000,-46.000000,-46.250000,-45.625000,-46.375000,-45.875000,-46.500000,-46.375000,...,-46.000000,-46.250000,-45.875000,-46.125000,-46.375000,-48.625000,-46.375000,-46.375000,-46.375000,-46.375000
TACGTAGGTGGCAAGCGTTGTCCGGATTTATTGGGCGTAAAGCGAGCGCAGGCGGTTCCTTAAGTCTGATGTGAAAGCCCACGGCTTAACCGTGGAAGGTCATTGGAAACTGGGGAACTTGAGTGCAGAAGAGGAGAGTGGAATTCCATGTGTAGCGGTGAAATGCGTAGATATATGGAGGAACACCAGTGGCGAAGGCGACTCTCTGGTCTGTAACTGACGCTGAGGCTCGAAAGCGTGGGGAGCAAACAGGATTAGATACCCTGGTAGTCCACGCCGTAAACGATGAGTGCTAAGTGTTGGAGGGTTTCCACCCTTCAGTGCTGCAGCTAACGCATTAAGCACTCCGCCTGGGGAGTACGGCCGCAAGGCTG,0.720588,0.802139,0.788770,0.692513,0.763369,0.707219,0.759358,0.727273,0.796791,0.740642,...,0.679144,0.729947,0.719251,0.708556,0.807487,0.766043,0.815508,0.794118,0.780749,0.692513
TACGTAGGGTGCGAGCGTTGTCCGGAATTATTGGGCGTAAAGAGCTTGTAGGCGGTTTGTCGCGTCTGCAGTGAAAATCCGGGGCTCAACCCCGGACTTGCTGTGGGTACGGGCAGACTAGAGTGTGGTAGGGGAGACCGGAATTCCTGGTGTAGCGGTGGAATGCGCAGATATCAGGAGGAACACCGATGGCGAAGGCAGGTCTCTGGGCCACAACTGACGCTGAGAAGCGAAAGCATGGGGAGCGAACAGGATTAGATACCCTGGTAGTCCATGCCGTAAACGTTGGGCGCTAGGTGTGGGGCTCATTCCACGAGCTCCGTGCCGCAGCTAACGCATTAAGCGCCCCGCCTGGGGAGTACGGCCGCAAGGCTA,0.701333,0.754667,0.744000,0.693333,0.722667,0.680000,0.724000,0.666667,0.912000,0.709333,...,0.704000,0.689333,0.713333,0.698667,0.744000,0.754667,0.778667,0.717333,0.736000,0.673333
TACGGAGGGTGCAAGCGTTATCCGGAATCACTGGGTTTAAAGGGTGCGTAGGCGGCTTAGTAAGTCAGTGGTGAAAGGCTGTGGCTTAACCATGGAATTGCCATTGATACTGCTGAGCTTGAATGAGGTTGAGGTTGGCGGAATGTGACATGTAGCGGTGAAATGCTTAGATATGTCATGGAACACCGATTGCGAAGGCAGCTGACTGGACCTATATTGACGCTGAGGCACGAAAGCGTGGGTAGCGAACAGGATTAGATACCCTGGTAGTCCACGCCCTAAACGATGCTTACTCGTTGTTTGATCGAAAGATTGAGTGACTAAGCGAAAGCGATAAGTAAGCCACCTGGGGAGTACGACCGCAAGGTTG,0.693243,0.701351,0.741892,0.664865,0.677027,0.808108,0.756757,0.797297,0.709459,0.678378,...,0.689189,0.795946,0.772973,0.706757,0.729730,0.737838,0.733784,0.695946,0.722973,0.695946
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
TACGGAGGGTGCAAGCGTTGTTCGGAATTACTGGGCGTAAAGCGCGTGTAGGCGGCTTGGTAAGTCGGATGTGAAATCCCTCGGCTTAACCGAGGACGTGCATCCGATACTGCTAGGCTTGAGTCTCGGAGAGGGAAGTGGAATTCCCGGTGTAGAGGTGAAATTCGTAGATATCGGGAGGAACACCAGTGGCGAAGGCGACTTCCTGGACGAAGACTGACGCTGAGACGCGAAAGCGTGGGGAGCAAACAGGATTAGATACCCTGGTAGTCCACGCCGTAAACGGTGTCAACTAGGTGTTGCGGGTATTGACCCCCGCAGTGCCGCAGCTAACGCATTAAGTTGACCGCCTGGGGAGTACGGCCGCAAGGTTA,0.756684,0.818182,0.815508,0.707219,0.762032,0.704545,0.804813,0.704545,0.780749,0.800802,...,0.680481,0.741979,0.691176,0.708556,0.815508,0.787433,0.807487,0.812834,0.810160,0.705882
TACAGAGGGTGCGAGCGTTAATCGGAATTACTGGGCGTAAAGGGCGCGTAGGCGGTGATATGAGTGTGATGTGAAAGCCCTGGGCTTAACCTAGGAAGTGCATCGCAAACGATATGACTGGAGTAGAGGAGAGGGTGGTGGAATTTCCGGTGTAGCAGTGAAATGCGTAGAGATCGGAAGGAACGTCGATGGCGAAG

In [ ]:
from numpy import array_split, ceil, load, save
import requests
from itertools import chain
import json
from numpy import isnan
import time
from tqdm import tqdm

def safe_isnan(x):
    return isinstance(x, float) and isnan(x)


# get the UIDs for all sequences
search_url = "https://eutils.ncbi.nlm.nih.gov/entrez/eutils/esearch.fcgi"
params = {
    "db": "assembly",
    "retmax": "100000",
    "retmode": "json",
    "api_key": "82683e91888a3832357e48d9a3ce9b9d6b08"
}

cut_midas_taxonomy = json.load(open("cut_midas_taxonomy.json", "r"))
best_levels = {"kingdom":[], "phylum": [], "class": [], "order": [], "family": [], "genus": [], "species": []}
insufficient_level = []
levels = best_levels.keys()

def get_uids(taxas, seq, threshold, sleep=5, pull=True):
    if (seq, threshold) in insufficient_level:
        return []
    IDs = []
    # print(len(taxas), "taxas to process")
    for taxa in taxas:
        if isinstance(taxa, str): taxa = cut_midas_taxonomy[taxa]
        # print(taxa)
        # previous_level = "kingdom"
        best_level = "phylum"
        for level in levels:
        # for level, name in list(taxa.items()):
            # print(level, name)
            if "midas" in taxa[level]: break
            # previous_level = best_level
            best_level = level
        best_levels[best_level].append(taxa[best_level])
        if best_level not in ["genus", "species"]:
            insufficient_level.append((seq, threshold))
            continue
        if not pull:  continue
        search_term = f"{taxa[best_level]}[Organism]"
        params.update({"term": search_term})
        r = requests.get(search_url, params=params).json()
        # print(r)
        if "esearchresult" not in r:
            if "API rate limit exceeded" in r["error"]:
                print(f"sleeping for {sleep} seconds with {taxa}")
                time.sleep(sleep)
                return get_uids(taxas, sleep=sleep+5)
            else:
                print(r)
                continue
        if "idlist" not in r["esearchresult"]:
            # print(r)
            # continue
            raise ValueError(f"Error in esearch result: {r}")
        IDs.append(r["esearchresult"]["idlist"])
    # print(IDs)
    return IDs

def uids_all_thresholds(seq):
    uids = {}
    seqIDmatches = seqID_matches[seq]
    for threshold in seqIDmatches:
        uids.setdefault(threshold, [])
        if (not safe_isnan(seq)
            and not safe_isnan(seqIDmatches[threshold])
            and threshold in seqIDmatches
            and len(seqIDmatches[threshold]) > 0):
            # print(seqID_matches[seq][threshold])
            uids[threshold] = get_uids(seqIDmatches[threshold], seq, threshold)
    json.dump(uids, open(f"uids/{seq}.json", "w"), indent=4)



from numpy import array_split, ceil, save, array
from itertools import chain


seqID_matches = json.load(open("MAG_matches.json", "r"))
# seqID_matches = json.load(open("seqID_matches.json", "r"))
seqIDs = load("seqIDs.npy", allow_pickle=True)

# insufficient_level = load("insufficient_sequences.npy")
missing_seqIDs = []
for seq in tqdm(seqIDs):
    uids = {}
    if seq not in seqID_matches:
        missing_seqIDs.append(seq)
        continue
    uids.setdefault(seq, {})
    for threshold in seqID_matches[seq]:
        uids[seq].setdefault(threshold, [])
        if (not safe_isnan(seq)
            and not safe_isnan(seqID_matches[seq][threshold])
            and threshold in seqID_matches[seq]
            and len(seqID_matches[seq][threshold]) > 0):
            uids[seq][threshold].extend(get_uids(seqID_matches[seq][threshold], seq, threshold, pull=False))
save("insufficient_sequences.npy", array(insufficient_level))

# from multiprocess import Pool
# from os import cpu_count

# cpus = int(cpu_count() * 2/3)+1
# pool = Pool(cpus) # 2/3 of the cores to not crash the computer
# args = [seq for seq in seqIDs]
# print(f"{cpus} cores are being used.  The first argument is {args[0]}")
# pool.map(uids_all_thresholds, args)

import os
for seq in tqdm(seqIDs):
    if os.path.exists(f"uids/{seq}.json"): continue
    # if seq not in seqID_matches:
    #     missing_seqIDs.append(seq)
    #     continue
    uids_all_thresholds(seq)
    # for threshold in seqID_matches[seq]:
    #     uids[seq].setdefault(threshold, [])
    #     if (not safe_isnan(seq)
    #         and not safe_isnan(seqID_matches[seq][threshold])
    #         and threshold in seqID_matches[seq]
    #         and len(seqID_matches[seq][threshold]) > 0):
    #         # print(seqID_matches[seq][threshold])
    #         uids[seq][threshold].extend(get_uids(seqID_matches[seq][threshold]))
        # if (not safe_isnan(seq) and threshold in seqID_matches[seq]):
        #     num_uids = min(num_uids, len(seqID_matches[seq][threshold]))
            # print(seq, seqID_matches[seq]["1.0"])
    # count += 1
    # if count == 5:
    #     break



# json.dump(uids, open("uids.json", "w"), indent=4)
# uids = list(chain.from_iterable([get_uids(seqID_matches[seq]["1.0"]) for seq in seqIDs if (not safe_isnan(seq) and "1.0" in seqID_matches[seq])][:num_uids]))
# print(f"Missing {len(missing_seqIDs)}/{len(seqIDs)} seqIDs")   # TODO investigate why these sequences are missing
# print(list(uids.items())[:2])

 60%|█████▉    | 5941/9959 [5:31:59<8:19:36,  7.46s/it] 

In [5]:
insufficient_level

array([], dtype=float64)

### after filtering at the genus level

In [ ]:
from collections import Counter

counted_entries = {}
for taxa, entries in best_levels.items():
    counted_entries[taxa] = Counter(entries)
# Counter(best_levels)
print("above genus", sum(list(counted_entries["kingdom"].values()
                              ) + list(counted_entries["phylum"].values()
                                       ) + list(counted_entries["class"].values()
                                                ) + list(counted_entries["order"].values()
                                                         ) + list(counted_entries["family"].values())))
print("at least genus", sum(list(counted_entries["genus"].values()
                                 ) + list(counted_entries["species"].values())))
display(counted_entries)

import json
json.dump(counted_entries, open("best_hit_levels.json", "w"), indent=4)

above genus 0
at least genus 3138749


{'kingdom': Counter(),
 'phylum': Counter(),
 'class': Counter(),
 'order': Counter(),
 'family': Counter(),
 'genus': Counter({'Flavobacterium': 122248,
          'Hyphomicrobium': 88198,
          'Comamonas': 87437,
          'Rhodobacter': 73711,
          'Novosphingobium': 72476,
          'Mesorhizobium': 70235,
          'Ferruginibacter': 47927,
          'Ottowia': 42689,
          'Legionella': 42650,
          'Paracoccus': 41716,
          'Rhodoferax': 41005,
          'Azonexus': 37210,
          'Hydrogenophaga': 34582,
          'Ca_Accumulibacter': 33180,
          'Devosia': 32738,
          'Terrimonas': 30539,
          'Leptothrix': 29941,
          'Acidovorax': 29812,
          'Ca_Competibacter': 29047,
          'Rubrivivax': 28434,
          'Limnohabitans': 26848,
          'Aminobacter': 26573,
          'Propionivibrio': 26067,
          'Pedomicrobium': 25678,
          'Pseudorhodobacter': 25220,
          'Christensenellaceae_R-7_group': 24844,
        

### without filtering at genus level

In [ ]:
from collections import Counter

counted_entries = {}
for taxa, entries in best_levels.items():
    counted_entries[taxa] = Counter(entries)
# Counter(best_levels)

print("above genus", sum(list(counted_entries["kingdom"].values()
                              ) + list(counted_entries["phylum"].values()
                                       ) + list(counted_entries["class"].values()
                                                ) + list(counted_entries["order"].values()
                                                         ) + list(counted_entries["family"].values())))
print("at least genus", sum(list(counted_entries["genus"].values()
                                 ) + list(counted_entries["species"].values())))
display(counted_entries)

import json
json.dump(counted_entries, open("best_hit_levels.json", "w"), indent=4)

above genus 1301661
at least genus 3138749


{'kingdom': Counter({'Bacteria': 137}),
 'phylum': Counter({'Desulfobacterota': 9053,
          'WPS-2': 1699,
          'Latescibacterota': 1512,
          'SAR324_cladeMarine_group_B': 1433,
          'Firmicutes': 1284,
          'Armatimonadota': 800,
          'WS1': 276,
          'WS2': 102,
          'MBNT15': 78,
          'LCP-89': 63,
          'Marinimicrobia_SAR406_clade': 59,
          'Margulisbacteria': 50,
          'Patescibacteria': 43,
          'FCPU426': 6,
          'Cyanobacteria': 6}),
 'class': Counter({'Gammaproteobacteria': 15277,
          'Actinobacteria': 9105,
          'Alphaproteobacteria': 7757,
          'OM190': 7083,
          'OLB14': 4561,
          'Lineage_IIb': 4003,
          'Anaerolineae': 2208,
          'KD4-96': 1003,
          'Acidimicrobiia': 841,
          'Planctomycetes': 723,
          'Chitinivibrionia': 692,
          'D8A-2': 666,
          'Clostridia': 658,
          'vadinHA49': 562,
          'Sericytochromatia': 524,
     

In [ ]:


# get the assemblies for all UIDs
def get_accessions(batch):
    summary_params = {
        "db": "assembly",
        "id": ",".join(batch),
        "retmode": "json"
    }
    summaries = requests.get(search_url, params=summary_params).json()["result"]
    accessions = []
    for uid in batch:
        record = summaries[uid]
        accessions.append({
            "accession": record["assemblyaccession"],
            "organism": record["organism"],
            "taxid": record["taxid"]
        })
    return accessions

num_batches = ceil(len(uids)/num_uids)
all_accessions = list(chain.from_iterable([get_accessions(batch) for batch in array_split(uids, num_batches)]))